In [1]:
import pprint
import os
import sys
import numpy as np
from params import get_cfg
from rnn_base import RNN
from mem_sel_rnn import MemRNN
from rnn_base import RNN
from history_rnn import HistoryRNN
from op_selector import OpSel
from mem_selector import MemSel
from ops import Operations
from session import *
from data_gen import samples_generator, split_train_test
import pickle
from data_gen import samples_generator, split_train_test

modelName = 'HistoryRNN'
func = 'np_avg_val-3ops'
name = 'state_size_50-num_samples_1500-batch_size_400-learning_rate_0.001-epsilon_0.001-loss_weight_0.5-num_features_5-norm_True-grad_norm_1000.0-softmax_sat_1-clip_False-share_state_True-rnns_same_state_False-seed_65774'
path = '/home/user/neural_program_synthesis/models/summaries/'+modelName+'/'+func+'/'+name+'/'
model_path = path+'/model'
#get the global configuration
cfg_path = path+'cfg.p'
cfg = pickle.load(open(cfg_path, 'rb'))
#instantiate containter with the operations avail for the selection
ops = Operations(cfg)
#generate data 
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], cfg['seed'])
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])

if modelName == "RNN":
        #instantiante the mem selection RNN
        mem = MemRNN(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = RNN(cfg, ops, mem)
        res = restore_selection_matrixes2RNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)
elif modelName == "HistoryRNN":
        #instantiante the mem and op selection
        mem_sel = MemSel(cfg, ops)
        op_sel = OpSel(cfg, ops)
        # instanitae the model graph with the main OP selection RNN
        model = HistoryRNN(cfg, ops, mem_sel, op_sel)
        res = restore_selection_matrixes_HistoryRNNS(model, cfg, x_train, x_test, y_train, y_test, model_path)

timestep 0
timestep 1
timestep 2
timestep 0
timestep 1
timestep 2
[<tf.Variable 'MemSel/Params/W_mem:0' shape=(55, 50) dtype=float64_ref>, <tf.Variable 'MemSel/Params/W2_mem:0' shape=(50, 4) dtype=float64_ref>, <tf.Variable 'OpSel/Params/W_op:0' shape=(55, 50) dtype=float64_ref>, <tf.Variable 'OpSel/Params/W2_op:0' shape=(50, 4) dtype=float64_ref>, <tf.Variable 'RNN_hist/Params/W_hist:0' shape=(150, 50) dtype=float64_ref>]
norming the grads
grads are
[<tf.Tensor 'RNN_hist/Grads/clip_by_global_norm/RNN_hist/Grads/clip_by_global_norm/_0:0' shape=(55, 50) dtype=float64>, <tf.Tensor 'RNN_hist/Grads/clip_by_global_norm/RNN_hist/Grads/clip_by_global_norm/_1:0' shape=(50, 4) dtype=float64>, <tf.Tensor 'RNN_hist/Grads/clip_by_global_norm/RNN_hist/Grads/clip_by_global_norm/_2:0' shape=(55, 50) dtype=float64>, <tf.Tensor 'RNN_hist/Grads/clip_by_global_norm/RNN_hist/Grads/clip_by_global_norm/_3:0' shape=(50, 4) dtype=float64>, <tf.Tensor 'RNN_hist/Grads/clip_by_global_norm/RNN_hist/Grads/clip_by_

In [2]:
def print_errors(error_lst, name):
    print("")
    for i, error in enumerate(error_lst):
        print(name + "[" + str(i) + "] err is " + str(error))

In [3]:
def print_matrix(total_error, matrix_lst):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            print(matrix[elem], end=" ")
       

In [4]:
def print_ops_matrix(total_error, matrix_lst, ops_list):
    np.set_printoptions(precision=3, suppress=True)
    print("Error for this matrix is", total_error)
    for elem in range(len(matrix_lst[0])):
        print("\n##Elem-"+str(elem)+"--#############################################################")        
        for matrix in matrix_lst:
            index = np.argmax(matrix[elem])
            op_name = ops_list[index].__name__
            print("[ "+op_name+" ]", end=" ")
            

In [5]:
print_errors(res["total_loss_traind_train"], "sofmax_train_error")
print_errors(res["total_loss_traind_test"], "hardmax_train_error")
print_errors(res["total_loss_testd_train"], "softmax_test_error")
print_errors(res["total_loss_testd_test"], "hardmax_test_error")


sofmax_train_error[0] err is 73.641406238
sofmax_train_error[1] err is 130.960327838

hardmax_train_error[0] err is 970696.628738
hardmax_train_error[1] err is 3.18676119752e+17

softmax_test_error[0] err is 5.7197298331e+17

hardmax_test_error[0] err is 1668111.50512


In [6]:
print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train"][1])
print_ops_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train"][1], ops.ops)

Error for this matrix is 130.960327838

##Elem-0--#############################################################
[ 0.063  0.194  0.457  0.286] [ 0.014  0.595  0.357  0.034] [ 0.096  0.001  0.001  0.901] 
##Elem-1--#############################################################
[ 0.15   0.298  0.404  0.148] [ 0.161  0.33   0.351  0.158] [ 0.515  0.001  0.001  0.483] 
##Elem-2--#############################################################
[ 0.099  0.236  0.458  0.208] [ 0.046  0.5    0.41   0.045] [ 0.231  0.001  0.002  0.766] 
##Elem-3--#############################################################
[ 0.026  0.38   0.546  0.049] [ 0.051  0.462  0.399  0.087] [ 0.118  0.001  0.001  0.881] 
##Elem-4--#############################################################
[ 0.017  0.435  0.534  0.015] [ 0.252  0.231  0.264  0.254] [ 0.414  0.002  0.003  0.581] 
##Elem-5--#############################################################
[ 0.067  0.4    0.479  0.054] [ 0.326  0.163  0.19   0.32 ] [ 0.496  0.00

In [9]:
print_matrix(res["total_loss_traind_test"][1], res["softmaxes_traind_test"][1])

Error for this matrix is 3.18676119752e+17

##Elem-0--#############################################################
[ 0.  0.  0.  1.] [ 0.  0.  1.  0.] [ 0.  0.  0.  1.] 
##Elem-1--#############################################################
[ 0.  0.  1.  0.] [ 0.  0.  1.  0.] [ 1.  0.  0.  0.] 
##Elem-2--#############################################################
[ 0.  0.  1.  0.] [ 0.  1.  0.  0.] [ 0.  0.  0.  1.] 
##Elem-3--#############################################################
[ 0.  0.  1.  0.] [ 0.  1.  0.  0.] [ 0.  0.  0.  1.] 
##Elem-4--#############################################################
[ 0.  0.  1.  0.] [ 0.  0.  1.  0.] [ 0.  0.  0.  1.] 
##Elem-5--#############################################################
[ 1.  0.  0.  0.] [ 0.  0.  1.  0.] [ 1.  0.  0.  0.] 
##Elem-6--#############################################################
[ 0.  0.  0.  1.] [ 0.  1.  0.  0.] [ 0.  0.  0.  1.] 
##Elem-7--#########################################################

In [8]:
print_matrix(res["total_loss_traind_train"][1], res["softmaxes_traind_train_mem"][1])
#print_ops_matrix(res["total_loss_traind_test"][0], res["softmaxes_traind_test"][0], ops.ops)

Error for this matrix is 130.960327838

##Elem-0--#############################################################
[ 0.001  0.992  0.001  0.005] [ 0.  1.  0.  0.] [ 0.002  0.993  0.002  0.003] 
##Elem-1--#############################################################
[ 1.  0.  0.  0.] [ 0.997  0.001  0.001  0.002] [ 0.999  0.     0.     0.001] 
##Elem-2--#############################################################
[ 0.  1.  0.  0.] [ 0.  1.  0.  0.] [ 0.     0.999  0.001  0.   ] 
##Elem-3--#############################################################
[ 0.     0.976  0.     0.023] [ 0.     0.966  0.     0.034] [ 0.004  0.757  0.001  0.237] 
##Elem-4--#############################################################
[ 1.  0.  0.  0.] [ 1.  0.  0.  0.] [ 1.  0.  0.  0.] 
##Elem-5--#############################################################
[ 0.999  0.     0.     0.   ] [ 0.999  0.     0.     0.   ] [ 1.  0.  0.  0.] 
##Elem-6--#############################################################
[ 0.  

In [ ]:
print_matrix(res["total_loss_traind_train"][1], res["outputs_traind_train_mem"][1])

In [ ]:
res["batchesX_train"][1]

In [ ]:
seed = 10000
x,y = samples_generator(cfg['train_fn'], (cfg['num_samples'], cfg['num_features']) , cfg['samples_value_rng'], seed)
x_train, x_test, y_train, y_test = split_train_test (x, y , cfg['test_ratio'])
res = predict_form_sess(model, cfg, x_test[0:100,], last_hardmax_state_train, model_path, mode="hard")

In [ ]:
res - y_test[0:100,]

In [ ]:
y_test[0:100,]

In [ ]:
res

In [ ]:
len(res["softmaxes_traind_train"])